<a href="https://colab.research.google.com/github/LuizGusQueiroz/Projetos_Estagio/blob/main/DREs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuração do ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Pré Processamento

In [ ]:
def get_dre(path):

  dre = pd.read_excel(path).fillna('')

  coluna = str()
  linha_cod = int()
  colum_cod = int()
  intervalos = list()
  indices = list()
  index_col = int()
  nomes_col = ['Código', 'Nome_cod']

  # Este laço encontra a coluna em que a palavra 'Código' está
  for col in dre:
    if 'Código' in list(dre[col]):
      coluna = col
      break

  # Encontra a linha da palavra 'Código'
  if coluna:
    linha_cod = dre[dre[coluna]=='Código'].index[0]
    colum_cod = dre.columns.get_loc(coluna)
  else:
    print('A palavra chave \'Código\' não foi encontrada')

  # Coleta os intervalos
  intervalos = get_intervalos(dre.iloc[linha_cod-1:linha_cod+2])
  # Armazena index das colunas que tem os valores.
  # dre.columns.get_loc(coluna) fornece o index da coluna que
  # tem a palavra 'Código'
  for col in dre.columns[dre.columns.get_loc(coluna)+2:]:
    # Converte o valor para string pois se apenas receber
    # o valor, valores 0 serão lidos como se não estivessem lá
    # pois retornam False
    valor = str(dre[col].iloc[linha_cod+2])
    if valor:
      indices.append(dre.columns.get_loc(col))
      if len(indices) == len(intervalos):
        break

  # Copia os valores das colunas que tem os valores para as n primeiras
  #colunas após o nome dos códigos
  for i,col in enumerate(dre.columns[dre.columns.get_loc(coluna)+2:]):
    dre[col] = dre[dre.columns[indices[i]]]
    i += 1
    if i == len(indices):
      break

  # Larga as colunas antes da coluna do Código    e após as cópias
  if colum_cod > 0:
    dre = dre.drop(dre.columns[0:colum_cod], axis='columns')
  # Larga as colunas após a última coluna de cópias
  # len(indices)+2 pega a coluna dos códigos, nomes códigos,
  # mais as n colunas com valores (len(indices))
  dre = dre.drop(dre.columns[len(indices)+2:], axis='columns')

  # Larga as linhas 0 até a linha que contém os intervalos inferiores
  dre = dre.drop(range(linha_cod+2), axis='index')

  # Atualiza o nome das colunas
  nomes_col.extend(intervalos)
  dre.columns = nomes_col

  return dre

In [ ]:
def get_intervalos(dre):
  # Converte a dre em lista e remove os valores '' com o if
  # Recebe apenas os valores com tamanho 7, pois é o formato das datas
  # há alguns valores ' ' que passam no teste se for apenas 'if i'
  # É feita a conversão para string pois alguns valores vem como float
  lower_bounds = [str(i) for i in list(dre.iloc[0]) if i and len(str(i))==7]
  upper_bounds = [str(i) for i in list(dre.iloc[2]) if i and len(str(i))==7]

  # Junta os limites em uma lista
  bounds = list()
  for i in range(len(lower_bounds)):
    bounds.append(f'{lower_bounds[i]} a {upper_bounds[i]}')

  return bounds

In [ ]:
def get_relacoes(path):
  # Lê o arquivo excel
  dre = pd.read_excel(path)
  # Filtra as colunas importantes para a relação
  dre = dre[['NomeDRE', 'CodDRE', 'CodigoFORTES']]
  # Padroniza os códigos iguais que diferem por um 0 no começo
  for i,valor in enumerate(dre['CodDRE']):
    # O (if valor) verifica valorez vazios, para evitar 'out of index'
    if valor and str(valor)[0] == '0':
      dre['CodDRE'][i] = str(valor)[1:]
  # Preenche os nulos com ''
  return dre.fillna('')

In [ ]:
def sort_intervalos(intervalos):
  # Os intervalos estão na forma mm/yyyy a mm/yyyy
  # então os índices [0,1] se referem ao mês
  # e os índices [3,4,5,6] se referem ao ano
  n = len(intervalos)
  # Orderna pelo ano
  for i in range(n):
    for j in range(i, n):
      # Compara os anos
      if int(intervalos[i][3:7]) > int(intervalos[j][3:7]):
        temp = intervalos[i]
        intervalos[i] = intervalos[j]
        intervalos[j] = temp
  # Ordena os meses com ano igual
  for i in range(n):
    for j in range(i, n):
      # Verifica anos iguais
      if intervalos[i][3:7] == intervalos[j][3:7]:
        # Compara os meses
        if int(intervalos[i][:2]) > int(intervalos[j][:2]):
          temp = intervalos[i]
          intervalos[i] = intervalos[j]
          intervalos[j] = temp

  return intervalos

In [ ]:
def init_final_dre(codigos, intervalos):
  rows = len(codigos) + 1
  cols = len(intervalos) + 1

  dre = pd.DataFrame(index=range(rows), columns=range(cols)).fillna(0)

  dre[0][0] = 'DRE Geral'

  for i in range(rows-1):
    dre[0][i+1] = codigos[i]

  for i in range(cols-1):
    dre[i+1][0] = intervalos[i]

  labels_col = ['Código']
  labels_col.extend(intervalos)

  dre.columns = labels_col

  return dre

In [ ]:
def sum_dre(dre_geral, dre, relacoes):
  for codigo_fortes in dre['Código']:
    if codigo_fortes in list(relacoes['CodigoFORTES']):
      codigo = list(relacoes['CodDRE'][relacoes['CodigoFORTES']==codigo_fortes])[0]
      # Linha da DRE
      index = dre[dre['Código']==codigo_fortes].index[0]
      # Linha que será atualizada na DRE geral
      index_g = dre_geral[dre_geral[dre_geral.columns[0]]==codigo].index[0]

      for periodo in dre.columns[2:]:
        # Há valores como string e outros como float, e alguns com ',' e outros com '.'
        # então é necessários converter para string, usar o replace para trocar ',' por '.'
        # e em seguida converter para float
        dre_geral[periodo][index_g] += float(str(dre[periodo][index]).replace(',', '.'))





  return dre_geral

# Salvando DRE's e Relações

In [ ]:
dre_condonal = get_dre('/content/drive/My Drive/DREs/DRE CONDONAL 102023.xls'           )
dre_fibra    = get_dre('/content/drive/My Drive/DREs/DRE FIBRA 092023.xls'              )
dre_ga       = get_dre('/content/drive/My Drive/DREs/DRE GA 092023.xls'                 )
dre_gestart  = get_dre('/content/drive/My Drive/DREs/DRE GESTART 092023.xls'            )
dre_hiseg    = get_dre('/content/drive/My Drive/DREs/DRE HISEG 092023 COMP.xls'         )
dre_natal    = get_dre('/content/drive/My Drive/DREs/DRE NATAL 092023.xls'              )
dre_primee   = get_dre('/content/drive/My Drive/DREs/DRE PRIMEE 092023.xls'             )
dre_solucao  = get_dre('/content/drive/My Drive/DREs/DRE SOLUÇÃO 092023 COMPETENCIA.xls')
dre_unica    = get_dre('/content/drive/My Drive/DREs/DRE UNICA 092023 COMP.xls'         )
dre_vigon    = get_dre('/content/drive/My Drive/DREs/DRE VIGON 092023.xls'              )

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [ ]:
relacoes_condonal = get_relacoes('/content/drive/My Drive/DREs/Relacoes/Relacoes_Condonal.xlsx')
relacoes_fibra    = get_relacoes('/content/drive/My Drive/DREs/Relacoes/Relacoes_Fibra.xlsx'   )
relacoes_ga       = get_relacoes('/content/drive/My Drive/DREs/Relacoes/Relacoes_Ga.xlsx'      )
relacoes_gestart  = get_relacoes('/content/drive/My Drive/DREs/Relacoes/Relacoes_Gestart.xlsx' )
relacoes_hiseg    = get_relacoes('/content/drive/My Drive/DREs/Relacoes/Relacoes_Hiseg.xlsx'   )
relacoes_natal    = get_relacoes('/content/drive/My Drive/DREs/Relacoes/Relacoes_Natal.xlsx'   )
relacoes_primee   = get_relacoes('/content/drive/My Drive/DREs/Relacoes/Relacoes_Primee.xlsx'  )
relacoes_solucao  = get_relacoes('/content/drive/My Drive/DREs/Relacoes/Relacoes_Solucao.xlsx' )
relacoes_unica    = get_relacoes('/content/drive/My Drive/DREs/Relacoes/Relacoes_Unica.xlsx'   )
relacoes_vigon    = get_relacoes('/content/drive/My Drive/DREs/Relacoes/Relacoes_Vigon.xlsx'   )

# Salvando todos os intervalos

In [ ]:
# Cria um set com todos os intervalos, que estão no
# nome das colunas de cada DRE
intervalos = set()
intervalos = intervalos.union(dre_condonal.columns)
intervalos = intervalos.union(dre_fibra   .columns)
intervalos = intervalos.union(dre_ga      .columns)
intervalos = intervalos.union(dre_gestart .columns)
intervalos = intervalos.union(dre_hiseg   .columns)
intervalos = intervalos.union(dre_natal   .columns)
intervalos = intervalos.union(dre_primee  .columns)
intervalos = intervalos.union(dre_solucao .columns)
intervalos = intervalos.union(dre_unica   .columns)
intervalos = intervalos.union(dre_vigon   .columns)
# Descarta os valores 'Código' e 'Nome_cod' que não são intervalos
intervalos.discard('Código')
intervalos.discard('Nome_cod')
# converter o set para lista
intervalos = list(intervalos)
# Ordena os intervalos
intervalos = sort_intervalos(intervalos)

# Salvando todos os códigos FORTES

In [ ]:
# Cria um set com todos os códigos de DRE presentes nas relações
codigos = set()
codigos = codigos.union(set(relacoes_condonal['CodDRE']))
codigos = codigos.union(set(relacoes_fibra   ['CodDRE']))
codigos = codigos.union(set(relacoes_ga      ['CodDRE']))
codigos = codigos.union(set(relacoes_gestart ['CodDRE']))
codigos = codigos.union(set(relacoes_hiseg   ['CodDRE']))
codigos = codigos.union(set(relacoes_natal   ['CodDRE']))
codigos = codigos.union(set(relacoes_primee  ['CodDRE']))
codigos = codigos.union(set(relacoes_solucao ['CodDRE']))
codigos = codigos.union(set(relacoes_unica   ['CodDRE']))
codigos = codigos.union(set(relacoes_vigon   ['CodDRE']))
# Converte o set para lista, o ordena e
# descarta um valor em branco ('') que entra como código
codigos = [i for i in codigos if i]
codigos.sort()

# DRE geral

In [ ]:
dre_geral = init_final_dre(codigos, intervalos)

<ipython-input-285-8ce853c8ba47>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dre[i+1][0] = intervalos[i]


In [ ]:
# ga, gestart, hiseg, primee

In [ ]:
dre_geral = sum_dre(dre_geral=dre_geral, dre=dre_condonal, relacoes=relacoes_condonal)
dre_geral = sum_dre(dre_geral=dre_geral, dre=dre_fibra   , relacoes=relacoes_fibra   )
#dre_geral = sum_dre(dre_geral=dre_geral, dre=dre_ga      , relacoes=relacoes_ga      )
#dre_geral = sum_dre(dre_geral=dre_geral, dre=dre_gestart , relacoes=relacoes_gestart )
#dre_geral = sum_dre(dre_geral=dre_geral, dre=dre_hiseg   , relacoes=relacoes_hiseg   )
dre_geral = sum_dre(dre_geral=dre_geral, dre=dre_natal   , relacoes=relacoes_natal   )
#dre_geral = sum_dre(dre_geral=dre_geral, dre=dre_primee  , relacoes=relacoes_primee  )
dre_geral = sum_dre(dre_geral=dre_geral, dre=dre_solucao , relacoes=relacoes_solucao )
dre_geral = sum_dre(dre_geral=dre_geral, dre=dre_unica   , relacoes=relacoes_unica   )
dre_geral = sum_dre(dre_geral=dre_geral, dre=dre_vigon   , relacoes=relacoes_vigon   )

In [ ]:
dre_geral

,Código,10/2022 a 10/2022,11/2022 a 11/2022,12/2022 a 12/2022,01/2023 a 01/2023,02/2023 a 02/2023,03/2023 a 03/2023,04/2023 a 04/2023,05/2023 a 05/2023,06/2023 a 06/2023,07/2023 a 07/2023,08/2023 a 08/2023,09/2023 a 09/2023,10/2023 a 10/2023
0,DRE Geral,10/2022 a 10/2022,11/2022 a 11/2022,12/2022 a 12/2022,01/2023 a 01/2023,02/2023 a 02/2023,03/2023 a 03/2023,04/2023 a 04/2023,05/2023 a 05/2023,06/2023 a 06/2023,07/2023 a 07/2023,08/2023 a 08/2023,09/2023 a 09/2023,10/2023 a 10/2023
1,1.01.001,15592711.1,15779489.91,15018515.19,18751197.46,18524010.27,19283265.06,20030047.86,20865999.12,21171227.03,21238119.96,21123572.36,21666026.1,4479055.88
2,1.01.002,272640.84,384758.17,368304.1,583633.81,422240.27,370888.4,323417.41,423346.4,384488.44,490995.71,380014.0,457139.21,38849.22
3,1.01.003,27745.01,128799.66,70146.01,19655.9,21177.39,0.0,0.0,6603.95,7652.12,33525.12,0.0,13741.56,0
4,1.01.004,73591.31,74450.83,73757.59,595671.91,592151.43,595164.96,587989.3,570052.87,566115.08,587726.82,585112.4,580743.98,531516.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,8.01.005,0,0,0,0,0,0,0,0,0,0,0,0,0
283,8.01.006,-26650.23,-25741.99,-25020.33,-24262.65,-25783.01,-24290.59,-22538.23,-71211.08,-64855.16,-77471.88,-282280.56,-91516.52,0
284,9.01.001,-9714.58,-10505.5,-10649.96,-17162.78,-13897.37,-13045.64,-10278.19,-22115.96,-26381.58,-35935.75,-20711.81,-16789.09,-7181.24
285,9.01.002,-2261.07,-3172.34,-4160.57,-3090.12,-2433.33,-5015.56,-1994.51,-138268.47,-3632.17,-3130.71,-1578.09,-1439.59,-995.38


In [ ]:
relacoes_solucao

,NomeDRE,CodDRE,CodigoFORTES
0,Locação Mão-Obra,1.01.001,101001
1,Diarias Extras,1.01.002,101002
2,Feriados,1.01.003,101003
3,Serviço Extra Administrativo,1.01.006,101004
4,INSS,2.01.001,201
...,...,...,...
217,Iptu.,6.01.034,07.001.031
218,Adiantamento Clientes,12.01.017,07.004.008
219,Estorno de Cheque,13.01.004,07.004.012
220,Estorno e Devolução de pagamento (Saida),13.01.009,07.004.016


In [ ]:
dre_solucao

,Código,Nome_cod,10/2022 a 10/2022,11/2022 a 11/2022,12/2022 a 12/2022,01/2023 a 01/2023,02/2023 a 02/2023,03/2023 a 03/2023,04/2023 a 04/2023,05/2023 a 05/2023,06/2023 a 06/2023,07/2023 a 07/2023,08/2023 a 08/2023,09/2023 a 09/2023
7,01,(=) Receita Bruta,4428813,4647263.34,4464335.36,4723655.38,4623856.38,5085263.19,5530839.11,5790402.44,5662130.87,5663268.46,5780766.16,5914065.2
8,01.001,Serviços Prestados,4428813,4647263.34,4464335.36,4723655.38,4623856.38,5085263.19,5530839.11,5790402.44,5662130.87,5663268.46,5780766.16,5914065.2
9,01.001.001,Locação Mão-Obra,4363334.11,4579765.75,4374647.36,4594421.96,4549565.9,4971525.05,5459583.47,5718189.74,5564509.05,5572518.98,5712853.8,5848509.84
10,01.001.002,Diarias Extras,64871.18,64840.81,86538.83,127686.2,72468.08,113078.15,68168.81,71001.55,96722.72,89362.29,66884.75,63995.47
11,01.001.004,Serviço Extra Administrativo,607.71,2656.78,3149.17,1547.22,1822.4,659.99,3086.83,1211.15,899.1,1387.19,1027.61,1559.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,16002,CSLL,0,0,0,-585.38,-609.26,-835.86,-570.97,-14062.32,-26414.59,-1486.53,0,0
145,16003,IRRF Retido,-424.87,-380.58,-380.01,-1114.55,-714.78,-1025.26,-1793.86,-1854.89,-2208.47,-1931.89,-994.1,-2060.04
146,16004,CSLL Retido,-23320.51,-25383.66,-24984.4,-26570.13,-26616.1,-33773.3,-30944.31,-35714.81,-36112.63,-36221.64,-37528.65,-37561.18
147,17,(=) Lucro Liquido,821711.69,-962712.94,823401.55,827091.15,986846.47,526211.24,1049267.3,1752991.35,1346907.04,655044.54,861470.95,1074764.65


In [ ]:
5136051.84+745066.69+3174197.97+2174060.49+4363334.11

15592711.100000001